# Download Sentinel-2 NIR band images from AWS and crop

Requires AWS and GDAL command line functionality.

In [20]:
import os
import subprocess
import numpy as np
import shutil
import geopandas as gpd
import fiona
import shapely
import glob

# Enable fiona KML file reading driver
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

In [21]:
# set basepath where the images will be downloaded
basepath = '/Volumes/JUKES_EXT/CautoRIFT_sites/S2_B3/'
# basepath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/'
# basepath = '/Users/jukesliu/Documents/GOI_Alaska/fieldmap/images/'

# 0) Automatically the identify S2 tile that overlaps your AOI

Must download the S2 tiling grid from here: https://sentinel.esa.int/web/sentinel/missions/sentinel-2/data-products

In [29]:
# AOIpath = '/Volumes/JUKES_EXT/AUTO-TERMINUS-TRACES/Antarctic-test/BoxThwaites/BufferThwaites_PS3031.shp' # path to the AOI rectangle shapefile
AOIpath = '/Users/jukesliu/Documents/PECLET/B3/B3/B3_Box_WGS.shp' 
# AOIpath = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/BoxTurner.shp' 
# set the path to the S2 tiling grid KML file:
S2grid_path = '/Users/jukesliu/Documents/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'

In [30]:
aoi_gdf = gpd.read_file(AOIpath) # read in the AOI shapefile
aoi = aoi_gdf.geometry.values
aoi

<GeometryArray>
Length: 1, dtype: geometry

In [31]:
s2grid_shp = fiona.open(S2grid_path) # open the S2 tile grid

# find the S2 foorptint tile(s) overlapping the AOI
tilename = 'None'
for feature in s2grid_shp:
    tile = shapely.geometry.Polygon(feature['geometry']['geometries'][0]['coordinates'][0])
#     print(feature['properties']['Name'])
    if tile.overlaps(aoi[0]):
        tilename=feature['properties']['Name']
        print(tilename)
        # break # stop searching
print(tilename)

33XXJ
35XMH
35XMH


In [32]:
# construct the folderpath (one at a time)
tilefolder = str(int(tilename[0:2]))+'/'+tilename[2:3]+'/'+tilename[3:]+'/'
print(tilefolder)

35/X/MH/


## Or set it manually here:

In [10]:
# tilefolder = '13/C/DS/'
tilefolder = '7/V/EG/'

## Explore files available on AWS manually:

In [33]:
# explore files manually:
!export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws s3 ls s3://sentinel-cogs/sentinel-s2-l2a-cogs/35/X/MH/2024/ --no-sign-request

                           PRE 3/
                           PRE 4/
                           PRE 5/
                           PRE 6/
                           PRE 7/
                           PRE 8/
                           PRE 9/


# 1) Set dates and download

Syntax:

aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/031/005/ Output/path/LS8aws/Path031_Row005/ --recursive --exclude "*" --include "*MTL.txt"

In [22]:
# SET YEARS AND MONTHS TO DOWNLOAD
years = np.arange(2013,2020); print(years) # set year(s) to download
months = np.linspace(1,12,12); print(months) # set month(s) to download
outputpath = basepath # outputpath will be in the basepath folder

[2022 2023 2024]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [4]:
# SET BAND
band = 'B08'

In [24]:
# loop through and download
for year in years:
    for month in months:
        year = str(year); month = str(int(month)) # convert to strings
        print('Downloading', year, month)

        cmd = 'export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; '
        cmd += 'aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/'+tilefolder+year+'/'+month+'/'
        cmd += ' '+outputpath+' --recursive --exclude "*/*" --include "*/'+band+'.tif"' # change the band here
        print(cmd)
        
        subprocess.run(cmd, check=True, shell=True)

export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/7/V/EG/2022/1/ /Volumes/JUKES_EXT/CautoRIFT_sites/S2_SK/ --recursive --exclude "*/*" --include "*/B08.tif"
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/7/V/EG/2022/1/S2A_7VEG_20220114_0_L2A/B08.tif to ../../../../../Volumes/JUKES_EXT/CautoRIFT_sites/S2_SK/S2A_7VEG_20220114_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/7/V/EG/2022/1/S2A_7VEG_20220124_0_L2A/B08.tif to ../../../../../Volumes/JUKES_EXT/CautoRIFT_sites/S2_SK/S2A_7VEG_20220124_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/7/V/EG/2022/1/S2B_7VEG_20220109_0_L2A/B08.tif to ../../../../../Volumes/JUKES_EXT/CautoRIFT_sites/S2_SK/S2B_7VEG_20220109_0_L2A/B08.tif
download: s3://sentinel-cogs/sentinel-s2-l2a-cogs/7/V/EG/2022/1/S2A_7VEG_20220127_0_L2A/B08.tif to ../../../../../Volumes/JUKES_EXT/CautoRIFT_sites/S2_SK/S2A_7VEG_20220127_0_L2A/B08.tif
download: s3://sentinel

# 2) Rename the files using acquisition and consolidate from subfolders:

In [25]:
for imgfolder in glob.glob(basepath+'S2*'):
    if not imgfolder.endswith('.tif') and not imgfolder.endswith('.xml'):
        files = os.listdir(imgfolder)
        for file in files:
            if file == band+'.tif': # rename the files (can change to different band if desired)
                spath = imgfolder+'/'+band+'.tif'
                tpath = basepath+imgfolder.split('/')[-1]+'_'+band+'.tif'
                os.rename(spath, tpath)  
                shutil.rmtree(imgfolder)

# 3) Crop images to AOI:

In [6]:
# set paths
import os
# imagepath = '/Volumes/JUKES_EXT/CautoRIFT_sites/S2_KG/' # path to the downloaded images
imagepath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/' # path to the downloaded images
# boxpath = AOIpath #  path to the AOI Box (UTM!!)
# imagepath = '/Users/jukesliu/Documents/GOI_Alaska/fieldmap/images/' # path to the downloaded images
boxpath = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/BoxTurner_UTM_07.shp'  #  path to the AOI Box (UTM!!)

In [7]:
# # REPROJECT FIRST:
# for image in os.listdir(imagepath): 
#     if not os.path.exists(imagepath+'reprojected/'+image): # if it hasn't already been reprojected
#         rp_cmd = 'gdalwarp -t_srs EPSG:32608 '+imagepath+image+' '+imagepath+'reprojected/'+image
#         os.system(rp_cmd)

In [8]:
# # crop with gdalwarp (command line) - REPROJECTED IMAGES
# for image in os.listdir(imagepath+'reprojected/'): 
#     # if it hasn't already been clipped
#     if not os.path.exists(imagepath+'reprojected/'+image[:-4]+'_clipped.tif') and image.endswith('.tif'):
#         crop_cmd = 'gdalwarp -cutline '+boxpath+' -crop_to_cutline '+imagepath+'reprojected/'+image+" "+imagepath+'reprojected/'+image[:-4]+'_clipped.tif'
#         print(crop_cmd)
#         os.system(crop_cmd)

...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220109__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220109__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220109__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220109__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220109__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220109__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220116__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220116__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220220__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220220__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220220__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220220__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220226__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220226__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220226__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220226__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220226__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220226__B08_clipped.tif.


...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220308__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220308__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220308__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220308__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220308__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220308__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220312__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220312__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif to destination /Volumes/SURGE_DIS

...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220421__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220421__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

...

Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08_clipped.tif.


...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221008__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221008__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif to destination /Volumes/SURGE_DIS

...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221102__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221102__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [13]:
# crop with gdalwarp (command line)
for image in os.listdir(imagepath): 
    # if it hasn't already been clipped
    if not os.path.exists(imagepath+image[:-4]+'_clipped.tif') and image.endswith('.tif'):
        if image.split('_')[2].startswith('202'):
            crop_cmd = 'gdalwarp -cutline '+boxpath+' -crop_to_cutline '+imagepath+image+" "+imagepath+image[:-4]+'_clipped.tif'
            print(crop_cmd)
            os.system(crop_cmd)

gdalwarp -cutline /Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/BoxTurner_UTM_07.shp -crop_to_cutline /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/S2A_7VEG_20210422_B08.tif /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/S2A_7VEG_20210422_B08_clipped.tif
Creating output file that is 4161P x 4140L.
Processing /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/S2A_7VEG_20210422_B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/S2A_7VEG_20210422_B08.tif.
Copying nodata values from source /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/S2A_7VEG_20210422_B08.tif to destination /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/S2A_7VEG_20210422_B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/BoxTurner_UTM_07.shp -crop_to_cutline /Users/jukesliu/Documents/TURNER/DATA/IMAGERY/senti